Notebook name reference: https://www.youtube.com/watch?v=drnBMAEA3AM

Used for lack of a better name...

# Species Distribution Modeling on Earth Engine

<table align="left">
 <td>
   <a href=_>
       <img src=https://cloud.google.com/ml-engine/images/colab-logo-32px.png alt="Colab logo">
     Run in Colab
   </a>
 </td>
 <td>
   <a href=_>
       <img src=https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32 alt=\"Vertex AI logo\">
     Open in Vertex AI Workbench
   </a>
 </td>
</table>
<br/><br/><br/>

In [ ]:
import ee
import geemap
import math
from IPython.display import JSON
from google.auth import compute_engine

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This allows for you to
# pull cloud credentials for auth with Earth Engine.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

        !pip install geemap -q

In [ ]:
# change to your cloud project
PROJECT = 'cloud-geographers-internal-gee'

In [ ]:
# authenticate with Earth Engine
scopes = [
    "https://www.googleapis.com/auth/earthengine"
]

credentials = compute_engine.Credentials(scopes=scopes)

ee.Initialize(
    credentials,
    project=PROJECT
)

In [ ]:
tracking_data = ee.FeatureCollection('projects/ee-kmarkert-demo/assets/GPS_tracking_bobcats_coyotes_WA')

In [ ]:
tracking_data = (
    tracking_data
    .map(lambda x: ee.Feature(x).set('millis', ee.Date(ee.Feature(x).get('system:time_start')).millis()))
)

In [ ]:
coyotes = tracking_data.filter(
  ee.Filter.eq("individual-taxon-canonical-name", "Canis latrans")
)
bobcat = tracking_data.filter(
  ee.Filter.eq("individual-taxon-canonical-name", "Lynx rufus")
)

In [ ]:
# display the
Map = geemap.Map()

Map.addLayer(coyotes, {"color":"red"}, "Coyotes")
Map.addLayer(bobcat, {"color":"blue"}, "Bobcat")

Map.centerObject(tracking_data, 10)

In [ ]:
#@title Choose your species

species = "Bobcat" #@param ["Bobcat", "Coyote"]


In [ ]:
if species.lower() == "bobcat":
    species_locations = bobcat.randomColumn().limit(5000,'random')
elif species.lower() == "coyote":
    species_locations = coyotes.randomColumn().limit(5000,'random')
else:
    raise ValueError("value for `species` not recongnized, options are 'Bobcat' or 'Coyote'")

In [ ]:
dates = (
    tracking_data
    .aggregate_array('system:time_start')
    .map(lambda x: ee.Date(x).format('YYYY-MM-01'))
    .distinct()
)

start_date = dates.reduce(ee.Reducer.min())
end_date = dates.reduce(ee.Reducer.max())


In [ ]:
# helper function to convert qa bit image to flag
def extract_bits(image, start, end=None, new_name=None):
    """Function to conver qa bits to binary flag image

    args:
        image (ee.Image): qa image to extract bit from
        start (int): starting bit for flag
        end (int | None, optional): ending bit for flag, if None then will only use start bit. default = None
        new_name (str | None, optional): output name of resulting image, if None name will be {start}Bits. default = None

    returns:
        ee.Image: image with extract bits
    """

    newname = new_name if new_name is not None else f"{start}Bits"

    if (start == end) or (end is None):
        # perform a bit shift with bitwiseAnd
        return image.select([0], [newname]).bitwiseAnd(1 << start)
    else:
        # Compute the bits we need to extract.
        pattern = 0
        for i in range(start, end):
            pattern += int(math.pow(2, i))

        # Return a single band image of the extracted QA bits, giving the band
        # a new name.
        return image.select([0], [newname]).bitwiseAnd(pattern).rightShift(start)

def preprocess_viirs(image):
    """Custom QA masking method for VIIRS VNP09GA dataset"""
    cloudMask = extract_bits(
        image.select("QF1"), 2, end=3, new_name="cloud_qa"
    ).lt(1)
    shadowMask = extract_bits(
        image.select("QF2"), 3, new_name="shadow_qa"
    ).Not()
    snowMask = extract_bits(image.select("QF2"), 5, new_name="snow_qa").Not()
    sensorZenith = image.select("SensorZenith").abs().lt(6000)

    qa_mask = cloudMask.And(shadowMask).And(sensorZenith)

    ndvi = image.normalizedDifference(['I2', 'I1']).rename('NDVI')

    return (
        image.select('(M|I).*')
        .addBands(ndvi)
        .updateMask(qa_mask)
    )



In [ ]:
viirs = (
    ee.ImageCollection("NOAA/VIIRS/001/VNP09GA")
    .filterDate(start_date, "2023-08-01")
    .map(preprocess_viirs)
)


In [ ]:
dem = ee.Image("NASA/NASADEM_HGT/001")
bioclim = ee.Image("WORLDCLIM/V1/BIO")

In [ ]:
absence_area = species_locations.geometry(1e4).bounds(1e4).difference(right = species_locations.geometry(1e2).buffer(500), maxError = 1e3)

In [ ]:
def sample_locations(date):
    """Function to sample locations from a date

    args:
        date (str): date in YYYY-MM-dd format

    returns:
        ee.FeatureCollection: sampled locations
    """
    start_date = ee.Date(date)
    end_date = start_date.advance(1, "month")

    tracks = species_locations.filter(
        ee.Filter.rangeContains('millis',start_date.millis(), end_date.millis())
    )

    presence = tracks.map(lambda x: ee.Feature(x).set('presence',1))

    absence = ee.FeatureCollection.randomPoints(absence_area, points=presence.size(), seed = start_date.millis(), maxError=1e3)
    absence = absence.map(lambda x: ee.Feature(x).set('presence',0))

    tracks = presence.merge(absence)

    sample_img = (
        viirs.filterDate(start_date, end_date).select("NDVI").mean()
        .addBands(dem.select('elevation'))
        .addBands(bioclim)
    )

    samples = sample_img.sampleRegions(
        collection=tracks,
        scale=1000,
        tileScale=16,
        geometries=True,
    )

    return samples

In [ ]:
samples = ee.FeatureCollection(dates.map(sample_locations)).flatten()

In [ ]:
# samples.first().getInfo()

In [ ]:
# task = ee.batch.Export.table.toAsset(samples, description='bobcat_sample_export', assetId=f"projects/{PROJECT}/assets/bobcat_samples")
# task.start()

In [ ]:
sample_fc = ee.FeatureCollection(f"projects/{PROJECT}/assets/bobcat_samples")

In [ ]:
img =  (
        viirs.filterDate("2019-04-01","2019-05-01").select("NDVI").mean().resample()
        .addBands(dem.select('elevation'))
        .addBands(bioclim).resample()
    )

In [ ]:
bobcat_classifier = (
    # ee.Classifier.smileRandomForest(20)
    ee.Classifier.amnhMaxent()
    .setOutputMode('PROBABILITY')
    .train(sample_fc,'presence', img.bandNames())
)

In [ ]:
from IPython.display import HTML
import json

In [ ]:
JSON(classifier_explained, root='Contributions')

In [ ]:
classifier_explained = bobcat_classifier.explain().getInfo()
classifier_explained


In [ ]:
x = img.classify(bobcat_classifier)

In [ ]:

Map.addLayer(x , {'bands':'probability'}, f'{species} Distribution')
# Map.addLayer(sample_fc,{'color':'red'}, 'Samples')